以下函数实现了将wordinfo中找到的信息，转化为anki的模板，写入到一个牌组中。但是缺少文章和例句的信息。同时实现了创建牌组的功能。
使用的时候需要打开anki。

In [ ]:
import requests
import sys
import time
from typing import Dict

# ====== 请替换为你实际的 get_word_info 导入或定义 ======
# from my_crawler_module import get_word_info
# ======================================================

ANKI_CONNECT_URL = "http://localhost:8765"
MODEL_NAME = "WordWithExamples"
REQUEST_TIMEOUT = 10.0

def invoke(action, **params):
    try:
        r = requests.post(
            ANKI_CONNECT_URL,
            json={"action": action, "version": 6, "params": params},
            timeout=REQUEST_TIMEOUT
        )
        r.raise_for_status()
        return r.json()
    except requests.RequestException as e:
        print(f"[错误] 无法连接 AnkiConnect（{ANKI_CONNECT_URL}）：{e}")
        sys.exit(1)

def model_exists(model_name: str) -> bool:
    res = invoke("modelNames")
    if res.get("error"):
        print("[错误] 调用 modelNames 出错：", res)
        return False
    return model_name in (res.get("result") or [])

def create_word_with_examples_model(model_name: str):
    """
    创建 Note Type：
    - 不再在模板顶部显示全局发音（避免重复）
    - 更大更清晰的字体
    - POS_Definitions 字段里包含每个词性的发音/释义/短语
    """
    css = """
.card {
  font-family: Arial, "Helvetica Neue", Helvetica, sans-serif;
  font-size: 16px;             /* 全局字体更大 */
  text-align: left;
  color: #111;
  background: white;
  line-height: 1.5;
  padding: 8px;
}
.word-header {
  font-size: 34px;             /* 单词标题更大 */
  text-align: center;
  margin: 8px 0 6px 0;
  font-weight: 600;
}
.pos-block {
  margin-bottom: 12px;
  font-size: 16px;
}
.pos-title {
  font-size: 18px;
  font-weight: 600;
  margin-bottom: 4px;
}
.definition-en {
  font-size: 15px;
}
.definition-ch {
  color: #666;
  font-size: 14px;
}
.phrase {
  font-style: italic;
}
.audio-row {
  margin: 4px 0;
  font-size: 14px;
}
"""

    front = r"""<div class="word-header">{{Word}}</div>"""

    back = r"""<div class="word-header">{{Word}}</div>
<hr>
{{#POS_Definitions}}
<div class="pos-block">{{POS_Definitions}}</div>
{{/POS_Definitions}}

{{#Examples}}
<div style="margin-top:8px;">
  <b>Examples</b>
  <div>{{Examples}}</div>
</div>
{{/Examples}}

{{#Tags}}
<div style="margin-top:8px; color:gray; font-size:12px;">Tags: {{Tags}}</div>
{{/Tags}}"""

    # 我们把字段简化为只需要这些（发音已并入 POS_Definitions）
    fields = ["Word", "POS_Definitions", "Examples", "Tags"]
    card_templates = [
        {
            "Name": "Card 1",
            "Front": front,
            "Back": back
        }
    ]

    print(f"正在创建模型: {model_name} ...")
    res = invoke(
        "createModel",
        modelName=model_name,
        inOrderFields=fields,
        css=css,
        cardTemplates=card_templates
    )
    if res.get("error"):
        print("[错误] createModel 返回错误：", res)
    else:
        print(f"模型 {model_name} 创建请求发送成功（Anki 端可能需要短暂时间应用）。")

def build_html_from_word_info(word_info: Dict) -> Dict[str, str]:
    """
    将 get_word_info 的结构转换为写入字段的 HTML 字符串，
    发音（audio）放在每个词性下面（不会在顶部重复显示）。
    返回字典包含 POS_Definitions、Examples（如有）。
    """
    pos_html_parts = []
    examples_parts = []

    for pos in word_info.get("partOfSpeech", []):
        pos_type = pos.get("type", "")
        part_lines = []
        # 标题
        part_lines.append(f"<div class='pos-title'>{pos_type.capitalize()}</div>")

        # 发音 (放在每个词性下面)
        uk = pos.get("pronunciationUK") or {}
        us = pos.get("pronunciationUS") or {}
        audio_lines = []
        if uk.get("phonetic") or uk.get("pronUrl"):
            aud = f"UK: {uk.get('phonetic','')}"
            if uk.get("pronUrl"):
                aud += f" <audio controls src=\"{uk.get('pronUrl')}\"></audio>"
            audio_lines.append(f"<div class='audio-row'>{aud}</div>")
        if us.get("phonetic") or us.get("pronUrl"):
            aud = f"US: {us.get('phonetic','')}"
            if us.get("pronUrl"):
                aud += f" <audio controls src=\"{us.get('pronUrl')}\"></audio>"
            audio_lines.append(f"<div class='audio-row'>{aud}</div>")
        if audio_lines:
            part_lines.extend(audio_lines)

        # 定义
        defs = pos.get("definitions") or []
        if defs:
            part_lines.append("<ul>")
            for d in defs:
                en = d.get("enMeaning", d.get("en", "")).strip()
                ch = d.get("chMeaning", d.get("ch", "")).strip()
                part_lines.append(
                    f"<li><div class='definition-en'>{en}</div>"
                    f"<div class='definition-ch'>{ch}</div></li>"
                )
            part_lines.append("</ul>")

        # 短语与短语释义
        phrases = pos.get("phrases") or []
        phrase_defs = pos.get("phraseDefinitions") or []
        if phrases:
            part_lines.append("<div><b>Phrases:</b><ul>")
            for i, ph in enumerate(phrases):
                pd = phrase_defs[i] if i < len(phrase_defs) else {}
                en = pd.get("enMeaning", pd.get("en", "")).strip()
                ch = pd.get("chMeaning", pd.get("ch", "")).strip()
                part_lines.append(
                    f"<li><span class='phrase'>{ph}</span> — <span class='definition-en'>{en}</span>"
                    f"<div class='definition-ch'>{ch}</div></li>"
                )
            part_lines.append("</ul></div>")

        # 合并该词性的 HTML
        pos_html_parts.append("<div>" + "\n".join(part_lines) + "</div>")

        # 如果有独立的例句数据，可加入 examples_parts（目前保留空）
        # examples_parts.append(...)

    pos_html = "\n".join(pos_html_parts)
    examples_html = "\n".join(examples_parts)

    return {
        "POS_Definitions": pos_html,
        "Examples": examples_html
    }

def add_word_to_anki(deck_name: str, word: str, word_info: Dict):
    fields = build_html_from_word_info(word_info)
    note = {
        "deckName": deck_name,
        "modelName": MODEL_NAME,
        "fields": {
            "Word": word,
            "POS_Definitions": fields.get("POS_Definitions", ""),
            "Examples": fields.get("Examples", ""),
            "Tags": ", ".join(word_info.get("tags", [])) if isinstance(word_info.get("tags", []), list) else (word_info.get("tags") or "")
        },
        "options": {"allowDuplicate": True},
        "tags": word_info.get("tags", []) or ["cambridge"]
    }

    res = invoke("addNote", note=note)
    # 若模型尚未被 Anki 识别，重试一次
    if res.get("error") and "model was not found" in str(res.get("error")):
        print("[警告] addNote 返回 model was not found，尝试等待并重试...")
        time.sleep(1.0)
        if not model_exists(MODEL_NAME):
            print("[错误] 模型仍然不存在，添加失败：", res)
            return res
        res = invoke("addNote", note=note)
    return res

def ensure_model_and_deck(deck_name: str):
    invoke("createDeck", deck=deck_name)
    if not model_exists(MODEL_NAME):
        create_word_with_examples_model(MODEL_NAME)
        # 等待模型被 Anki 端应用
        for i in range(10):
            time.sleep(0.4)
            if model_exists(MODEL_NAME):
                print(f"模型 {MODEL_NAME} 已创建并可用。")
                break
        else:
            print(f"[错误] 等待模型创建超时，请检查 Anki 端是否有错误提示。")
    else:
        print(f"模型 {MODEL_NAME} 已存在，跳过创建。")

# ----------------- 主流程示例 -----------------
if __name__ == "__main__":
    deck = "CambridgeDeck"
    ensure_model_and_deck(deck)

    # 示例：用你的 get_word_info 函数
    try:
        word = "spare"
        word_info = get_word_info(word)  # 请确保 get_word_info 在当前作用域可用
    except NameError:
        print("[提示] 未找到 get_word_info 函数，请将顶部替换为你的导入或定义。")
        sys.exit(1)

    print(f"导入单词 {word} ...")
    result = add_word_to_anki(deck, word, word_info)
    print("AnkiConnect 返回：", result)


模型 WordWithExamples 已存在，跳过创建。
导入单词 spare ...
AnkiConnect 返回： {'result': 1758159396030, 'error': None}
